In [1]:
import nltk
from os import getcwd

filepath = f"{getcwd()}/../tmp2/"
nltk.data.path.append(filepath)

import numpy as np
import pandas as pd
import re
import string

from nltk.stem import PorterStemmer
from nltk.corpus import stopwords

from nltk.tokenize import TweetTokenizer #keep tweet tokenizer for now since good for general "casual" reviews


In [2]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/emmabarnes/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [27]:
#data data
illicit_reviews_input = '/Users/emmabarnes/Documents/data for testing/illicit_reviews.xlsx'
nonillicit_reviews_input = '/Users/emmabarnes/Documents/data for testing/nonillicit_reviews.xlsx'

df1 = pd.read_excel(illicit_reviews_input)
df2 = pd.read_excel(nonillicit_reviews_input)

illicit_reviews = df1['REVIEW'].tolist()
nonillicit_reviews = df2['REVIEW'].tolist()

In [28]:
print(len(nonillicit_reviews))

1000


In [29]:
test_illicit = illicit_reviews[:200]
train_illicit = illicit_reviews[200:]
test_nonillicit = nonillicit_reviews[:200]
train_nonillicit = nonillicit_reviews[200:]

train_x = train_illicit + train_nonillicit
test_x = test_illicit + test_nonillicit

In [30]:
train_y = np.append(np.ones((len(train_illicit), 1)), np.zeros((len(train_nonillicit), 1)), axis = 0)
test_y = np.append(np.ones((len(test_illicit), 1)), np.zeros((len(test_nonillicit), 1)), axis = 0)

In [31]:
print("train_y.shape = " + str(train_y.shape))
print("test_y.shape = " + str(test_y.shape))

train_y.shape = (1600, 1)
test_y.shape = (400, 1)


dict

In [3]:
def process_tweet(tweet):
    stemmer = PorterStemmer()
    stopwords_english = stopwords.words('english')
    # remove stock market tickers like $GE
    tweet = re.sub(r'\$\w*', '', tweet)
    # remove old style retweet text "RT"
    tweet = re.sub(r'^RT[\s]+', '', tweet)
    # remove hyperlinks
    tweet = re.sub(r'https?:\/\/.*[\r\n]*', '', tweet)
    # remove hashtags
    # only removing the hash # sign from the word
    tweet = re.sub(r'#', '', tweet)
    # tokenize tweets
    tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True, reduce_len=True)
    tweet_tokens = tokenizer.tokenize(tweet)

    tweets_clean = []
    for word in tweet_tokens:
        if (word not in stopwords_english and word not in string.punctuation): 
        # tweets_clean.append(word)
            stem_word = stemmer.stem(word)  # stemming word
            tweets_clean.append(stem_word)
    return tweets_clean

In [4]:
def build_freqs(tweets, ys):
    # Convert np array to list since zip needs an iterable.
    # The squeeze is necessary or the list ends up with one element.
    # Also note that this is just a NOP if ys is already a list.
    yslist = np.squeeze(ys).tolist()

    # Start with an empty dictionary and populate it by looping over all tweets
    # and over all processed words in each tweet.
    freqs = {}
    for y, tweet in zip(yslist, tweets):
        for word in process_tweet(tweet):
            pair = (word, y)
            if pair in freqs:
                freqs[pair] += 1
            else:
                freqs[pair] = 1

    return freqs

In [32]:
freqs = build_freqs(train_x, train_y)

In [35]:
print(str(type(freqs)))
print(len(freqs.keys()))

<class 'dict'>
9077


In [36]:
print(train_x[0])

I haven’t seen her here before. She looked good, really young petite body, amazing tits, and just a radiant positive attitude. Her face is really cute too, and she’s down to fuck which is exactly what I wanted. 

When she was working the massage I could feel she had some experience. Her hands felt good as she was taking care of those sore areas. When she flipped me she saw I was getting a boner and started playing around with my cock while giving me tongue kisses. I asked her for the pussy and we got to talking. 

She took off her little white dress and started with a BBBJ. She went in deep and slobbered all over my pole. Good suction, and great hand coordination. Moved her into 69, and licked on her pussy until we were both ready to fuck. 

I got the rubber on and she got on top, squatted down, and inserted my cock in her tight pussy. I loved fucking her, and making out with her as she rode me Cowgirl like a girlfriend. Pushed her up to admire her bare chest while she just grinded awa

In [37]:
print(process_tweet(train_x[0]))

['’', 'seen', 'look', 'good', 'realli', 'young', 'petit', 'bodi', 'amaz', 'tit', 'radiant', 'posit', 'attitud', 'face', 'realli', 'cute', '’', 'fuck', 'exactli', 'want', 'work', 'massag', 'could', 'feel', 'experi', 'hand', 'felt', 'good', 'take', 'care', 'sore', 'area', 'flip', 'saw', 'get', 'boner', 'start', 'play', 'around', 'cock', 'give', 'tongu', 'kiss', 'ask', 'pussi', 'got', 'talk', 'took', 'littl', 'white', 'dress', 'start', 'bbbj', 'went', 'deep', 'slobber', 'pole', 'good', 'suction', 'great', 'hand', 'coordin', 'move', '69', 'lick', 'pussi', 'readi', 'fuck', 'got', 'rubber', 'got', 'top', 'squat', 'insert', 'cock', 'tight', 'pussi', 'love', 'fuck', 'make', 'rode', 'cowgirl', 'like', 'girlfriend', 'push', 'admir', 'bare', 'chest', 'grind', 'away', 'want', 'doggi', 'got', 'knee', 'fuck', 'behind', 'till', 'blew', 'load', 'bag', 'great', 'time', 'sure', 'left', 'well', 'deserv', 'tip', 'outta', 'feel', 'satisfi']


In [38]:
def sigmoid(z):
    h = 1/(1+np.exp(-z))
    return h

In [39]:
def gradient (x, y, theta, alpha, num_iters):
    m = len(y)
    for i in range(0, num_iters):
        z = np.dot(x, theta)
        h = sigmoid(z)
        J = (-1/m)*np.sum(y*np.log(h)+(1-y)*np.log(1-h))
        theta = theta-(alpha/m)*np.dot(x.T, (h-y))
    J = float(J)
    return J, theta

In [40]:
#check
np.random.seed(1)
#X input is 10 by 3 dim with 1s for biases
tmp_X = np.append(np.ones((10,1)), np.random.rand(10,2)*2000, axis=1)
#Y labels are 10 by 1
tmp_Y = (np.random.rand(10,1)>0.35).astype(float)

#with gradient
tmp_J, tmp_theta = gradient(tmp_X, tmp_Y, np.zeros((3,1)), 1e-8, 700)
print("cost" + str(tmp_J))
print("weight vector " + str([round(t, 8) for t in np.squeeze(tmp_theta)]))

cost0.6709497038162117
weight vector [4.1e-07, 0.00035658, 7.309e-05]


In [41]:
def extractFeatures(tweet, freqs, process_tweet=process_tweet):
    word_l = process_tweet(tweet)
    x = np.zeros(3)
    x[0] = 1
    
    for word in word_l:
        x[1] += freqs.get((word, 1), 0) #positive label
        x[2] += freqs.get((word, 0), 0) #neg label
    x = x[None, :]
    assert(x.shape == (1,3))
    return x

In [42]:
#first test on training data
tmp1 = extractFeatures(train_x[0], freqs)
print(tmp1)

[[1.0000e+00 2.6382e+04 1.7974e+04]]


In [43]:
X = np.zeros((len(train_x), 3)) #collects features x and stacks to matrix X

for i in range(len(train_x)):
    X[i, :]= extractFeatures(train_x[i], freqs)
    
Y = train_y #training labels for X

#apply gradient
J, theta = gradient(X, Y, np.zeros((3, 1)), 1e-9, 1500)

print("cost of training" + str(J))
print("vector weights" + str([round(t, 8) for t in np.squeeze(theta)]))

cost of training0.5909053105765716
vector weights[-5e-08, 0.0002452, -0.00028874]


In [44]:
def predict(tweet, freqs, theta):
    x = extractFeatures(tweet, freqs)
    y_pred = sigmoid(np.dot(x, theta))
    
    return y_pred

In [55]:
#testing output
for tweet in ['I got a hj','i got fs', 'i did not get fs','i got a handjob','I did not get a handjob', 'this sucks but i got a BBJ', 'this is great and i got an HJ', 'fuck', 'hj hj hj', 'legit, no luck here', 'no hj here']:
    print(str(tweet) + " === " + str(predict(tweet, freqs, theta)))

I got a hj === [[0.51181341]]
i got fs === [[0.51204864]]
i did not get fs === [[0.50615208]]
i got a handjob === [[0.50869168]]
I did not get a handjob === [[0.50279395]]
this sucks but i got a BBJ === [[0.52069349]]
this is great and i got an HJ === [[0.51916714]]
fuck === [[0.51050219]]
hj hj hj === [[0.50989552]]
legit, no luck here === [[0.49177732]]
no hj here === [[0.50329888]]


In [56]:
def testAccuracy (test_x, test_y, freqs, theta, predict=predict):
    y_hat = []
    for tweet in test_x:
        y_pred = predict(tweet, freqs, theta)
        
        if y_pred > 0.5: #good prediction
            y_hat.append(1.0)
        else:
            y_hat.append(0.0)
            
    accuracy = np.sum(y_hat==test_y.squeeze())/len(test_x)#merge into a 1d array with == to compare
    
    return accuracy

In [57]:
tmp_accuracy = testAccuracy(test_x, test_y, freqs, theta)
print("accuracy: " + str(tmp_accuracy))

accuracy: 0.7425


In [66]:
#error analysis
print("pred review======")
for x, y in zip(test_x, test_y):
    y_hat = predict(x, freqs, theta)
    if np.abs(y-(y_hat > 0.5)) > 0:
        if(y_hat > 0.5):
            print("the review is: " , x)
            print("te processed review is: ", process_tweet(x))
            print('%d\t%0.8f\t%s' % (y, y_hat, ' '.join(process_tweet(x)).encode('ascii', 'ignore')))

pred review======
the review is:  I went shopping and after the shopping was done I saw this place driving by and wanted to check it out. Walked in and was greeted by the MMS, and she sent me to a room Sunny was cute.

The massage was well handled she did a good job on my lower back and my arms were killing me and she worked them out too.

when we came to the flip. She hadn't given any signs of a HJ so I just signaled, and she said no, but you can do it. WHAT? I didn't come here to jerk off to a full-clothed woman. I played it cool though and acted like it was nothing. 2 minutes later, she was like OK, and she began to stroke my cock and it was really good, I guess she needed the cash so she really did a number on my johnson and I finished all over her hands, it was nice.
te processed review is:  ['went', 'shop', 'shop', 'done', 'saw', 'place', 'drive', 'want', 'check', 'walk', 'greet', 'mm', 'sent', 'room', 'sunni', 'cute', 'massag', 'well', 'handl', 'good', 'job', 'lower', 'back', 'a